# **Image Classification with CNN for Malaria Data**

In this project I will predict whether a cell is infected or uninfected from Malaria Dataset. I am going to use CNN-Keras for this mission. My data has 27650 pictures and 2 classes.

In [ ]:
import pandas as pd
import os

In [10]:
#I am unziping the dataset
from google.colab import files

import zipfile

uploaded = files.upload()

for zip_filename in uploaded.keys():
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('Cell_Image_Dataset')

Saving cell-images-for-detecting-malaria.zip to cell-images-for-detecting-malaria.zip


In [29]:
#I am writing the path of the images
#I am writing the labels
labels=["Parasitized","Uninfected"]
img_path="/content/Cell_Image_Dataset/cell_images/"

In [30]:
#I am reading the images paths and labels
img_list=[]
label_list=[]
for label in labels:
    for img_file in os.listdir(img_path+label):
        img_list.append(img_path+label+'/'+img_file)
        label_list.append(label)

In [31]:
#I am inserting the paths and labels in a dataframe
df=pd.DataFrame({'img':img_list,'label':label_list})

In [32]:
df.head()

,img,label
0,/content/Cell_Image_Dataset/cell_images/Parasi...,Parasitized
1,/content/Cell_Image_Dataset/cell_images/Parasi...,Parasitized
2,/content/Cell_Image_Dataset/cell_images/Parasi...,Parasitized
3,/content/Cell_Image_Dataset/cell_images/Parasi...,Parasitized
4,/content/Cell_Image_Dataset/cell_images/Parasi...,Parasitized


In [33]:
df.shape

(27560, 2)

In [34]:
#I am encoding the labels
d={'Parasitized':1,'Uninfected':0}

In [35]:
df["encoded_label"]=df["label"].map(d)

In [37]:
#From ChatGPT
#That is the code to resize the images to 170x170
#It can understand that can the data resize and normalize
#If the image is not readable, it won't be in x list.
import cv2
import pandas as pd
import os

x = []
processed_paths = []

for img_path in df["img"]:
    try:
        img = cv2.imread(str(img_path))
        if img is None:
            raise FileNotFoundError(f"{img_path} file didn't read.")

        img = cv2.resize(img, (170, 170))
        img = img / 255.0
        x.append(img)
        processed_paths.append(img_path)
    except Exception as e:
        print(f"Error: {e}")

        if os.path.exists(img_path):
            os.remove(img_path)
        else:
            print(f"Attention: {img_path} file has already been deleted or cannot be found.")




Hata: /content/Cell_Image_Dataset/cell_images/Parasitized/Thumbs.db dosyası okunamadı.
Hata: /content/Cell_Image_Dataset/cell_images/Uninfected/Thumbs.db dosyası okunamadı.


In [38]:
#I am converting the list to an array
x=np.array(x)

In [39]:
#I am creating my target
y=df["encoded_label"]

In [44]:
#I am finding the image index of the unread
df[df["img"]=="/content/Cell_Image_Dataset/cell_images/Parasitized/Thumbs.db"]

,img,label,encoded_label
12909,/content/Cell_Image_Dataset/cell_images/Parasi...,Parasitized,1


In [46]:
#I am deleting it from the target
y.drop(12910, inplace=True)

In [47]:
#I am finding the image index of the unread
df[df["img"]=="/content/Cell_Image_Dataset/cell_images/Uninfected/Thumbs.db"]

,img,label,encoded_label
26614,/content/Cell_Image_Dataset/cell_images/Uninfe...,Uninfected,0


In [48]:
#I am deleting it from the target
y.drop(26615, inplace=True)

In [49]:
from sklearn.model_selection import train_test_split

In [50]:
#Train test split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

# **Modelling**

In [51]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, Input, MaxPooling2D, Dropout, BatchNormalization

In [52]:
#My model is a convolutional neural network
#I am using 5 convolutional layers
#I am using 3 maxpooling layers
#I am using 2 dense layers
model=Sequential()
model.add(Input(shape=(170,170,3)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(50,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(68,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(90,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [53]:
history=model.fit(x_train,y_train,epochs=15,validation_data=(x_test,y_test))

Epoch 1/15
689/689 [==============================] - 122s 175ms/step - loss: 0.2429 - accuracy: 0.9027 - val_loss: 0.1991 - val_accuracy: 0.9233
Epoch 2/15
689/689 [==============================] - 119s 173ms/step - loss: 0.1639 - accuracy: 0.9482 - val_loss: 0.1480 - val_accuracy: 0.9572
Epoch 3/15
689/689 [==============================] - 119s 173ms/step - loss: 0.1502 - accuracy: 0.9532 - val_loss: 0.1445 - val_accuracy: 0.9572
Epoch 4/15
689/689 [==============================] - 119s 173ms/step - loss: 0.1376 - accuracy: 0.9551 - val_loss: 0.1314 - val_accuracy: 0.9590
Epoch 5/15
689/689 [==============================] - 119s 172ms/step - loss: 0.1287 - accuracy: 0.9557 - val_loss: 0.1353 - val_accuracy: 0.9606
Epoch 6/15
689/689 [==============================] - 120s 174ms/step - loss: 0.1217 - accuracy: 0.9584 - val_loss: 0.1309 - val_accuracy: 0.9566
Epoch 7/15
689/689 [==============================] - 120s 175ms/step - loss: 0.1125 - accuracy: 0.9607 - val_loss: 0.1229 -

In [54]:
#I am saving my model
model.save("my_malaria_prediction_model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


To conclusion I can say that the model is good. It works and predicts very well. You can find data here [Original Website](https://ceb.nlm.nih.gov/repositories/malaria-datasets/
) or [Kaggle](https://www.kaggle.com/datasets/iarunava/cell-images-for-detecting-malaria)